# ngrams

> Functionality for ngram analysis.

In [ ]:
#| default_exp ngrams

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import numpy as np
import time
import polars as pl
from spacy.attrs import ORTH, LOWER # remove? - add ENT_TYPE, ENT_IOB
import math
from fastcore.basics import patch

In [ ]:
#| export
from conc.corpus import Corpus
from conc.result import Result
from conc.core import logger, PAGE_SIZE

In [ ]:
#| export
class Ngrams:
	""" Class for n-gram analysis reporting. """
	def __init__(self,
			  corpus:Corpus # Corpus instance
			  ): 
		self.corpus = corpus


In [ ]:
#| exporti
@patch
def _get_ngrams(self:Ngrams, 
			   token_sequence: list[np.ndarray], # token sequence to get index for 
			   index_id: int, # index to search (i.e. ORTH, LOWER)
			   token_index: list[np.ndarray], # positions of token sequence, returned by get_token_index 
			   ngram_length: int = 2, # length of ngram
			   ngram_word_position: str = 'LEFT' # specify if token sequence is on LEFT, RIGHT, or MIDDLE of ngrams
			   ) -> np.ndarray: # array of ngrams results

	""" Get ngram data for a token sequence. """
	
	start_time = time.time()
	sequence_len = len(token_sequence[0])
	variants_len = len(token_sequence)
	token_index_len = len(token_index[0])

	if index_id == ORTH:
		index = 'orth_index'
	else:
		index = 'lower_index'

	if ngram_word_position == 'RIGHT':
		ngram_range = range(-1 * ngram_length + sequence_len, sequence_len)
	elif ngram_word_position == 'MIDDLE':
		ngram_range = range(-1 * ngram_length + sequence_len + 1, sequence_len + 1)
	else:
		ngram_range = range(0, ngram_length)

	ngrams = []
	for pos in ngram_range:
		if variants_len == 1 and pos > -1 and pos < sequence_len:
			ngrams.append(np.full(token_index_len, token_sequence[0][pos]))
		else:
			seq = token_index[0] + pos
			ngrams.append(getattr(self.corpus, index)[seq])

	ngrams = np.stack(ngrams)

	# getting positions to search for EOF_TOKEN and filter out ngrams crossing doc boundaries
	positions = (np.array(ngram_range)[:, None] != np.arange(sequence_len)).all(axis=1)
	ngrams = np.delete(ngrams, np.where(ngrams[positions] == self.corpus.EOF_TOKEN)[1], axis=1)

	logger.info(f'Ngrams ({ngrams.shape[1]}) retrieval time: {(time.time() - start_time):.5f} seconds')
	return ngrams


In [ ]:
#| hide
path_to_corpus_file = '../test-corpora/saved/brown.corpus'

In [ ]:
# load the corpus
brown = Corpus().load(path_to_corpus_file)

In [ ]:
# instantiate the Ngrams class
report_brown = Ngrams(brown)

In [ ]:
#| hide
token_str = 'dog'
brown_token_sequence, brown_index_id = brown.tokenize(token_str, simple_indexing=True)
brown_token_index = brown.get_token_index(brown_token_sequence, brown_index_id)
ngrams = report_brown._get_ngrams(brown_token_sequence, brown_index_id, brown_token_index, ngram_length = 2, ngram_word_position = 'LEFT')
print(ngrams)

[[23289 23289 23289 23289 23289 23289 23289 23289 23289 23289 23289 23289
  23289 23289 23289 23289 23289 23289 23289 23289 23289 23289 23289 23289
  23289 23289 23289 23289 23289 23289 23289 23289 23289 23289 23289 23289
  23289 23289 23289 23289 23289 23289 23289 23289 23289 23289 23289 23289
  23289 23289 23289 23289 23289 23289 23289 23289 23289 23289 23289 23289
  23289 23289 23289 23289 23289 23289 23289 23289 23289 23289 23289 23289
  23289 23289 23289 23289 23289 23289 23289]
 [29064 38309 33838 15829   795 47534  9374 42833 36136 21550 17435 43533
  25158  8128 11957  5963  5280 14413 16089  8128 51356 51356 13607 15531
   1147  5280 27404  1916  5963 38309  5280 33838 20463 38309 49761 27963
  55470 55470 27831  9374  8128 38309 38309 38309  8128  9374  8128 30252
  35250 32807 49398 38309  7126 43244 38309  8128  8128 38309  9374  8128
  38309 22848 34812 21550 49732 27963  8128 13462 38382 38309 49732 42833
  55687 38309 15800  3883 38309 46196 14474]]


In [ ]:
#| export
@patch
def ngrams(self: Ngrams, 
		   token_str: str, # token string to get ngrams for 
		   ngram_length:int = 2, # length of ngram
		   ngram_word_position:str = 'LEFT', # specify if token sequence is on LEFT, RIGHT, or MIDDLE of ngrams
		   page_size:int = PAGE_SIZE, # number of results to display per results page 
		   page_current:int = 0, # current page of results
		   show_all_columns:bool = False, # return raw df with all columns or just ngram and frequency
		   use_cache:bool = True # retrieve the results from cache if available
		   ) -> Result: # return a Result object with ngram data
	""" Report ngrams for a token string. """

	token_sequence, index_id = self.corpus.tokenize(token_str, simple_indexing=True)

	start_time = time.time()
	cache_id = tuple(['ngram'] + list(token_sequence) + [ngram_length, ngram_word_position])

	if use_cache == True and cache_id in self.corpus.results_cache:
		logger.info('Using cached ngrams results')
		ngrams_report = self.corpus.results_cache[cache_id][0]
		total_unique = self.corpus.results_cache[cache_id][1]
		total_count = self.corpus.results_cache[cache_id][2]
	else:
		token_index = self.corpus.get_token_index(token_sequence, index_id)
		
		if len(token_index[0]) == 0:
			logger.info('No tokens found')
			return None, {}, []

		logger.info('Generating ngrams results')
		ngrams = self._get_ngrams(token_sequence, index_id, token_index, ngram_length = ngram_length, ngram_word_position = ngram_word_position)
		total_count = ngrams.shape[1]
		schema = [f'token_{i+1}' for i in range(ngram_length)]
		ngrams_report = pl.DataFrame(ngrams.T, schema=schema).to_struct(name = 'ngram_token_ids').value_counts(sort=True).rename({"count": "frequency"})
		ngrams_report = ngrams_report.with_row_index(name='rank', offset=1)
		total_unique = len(ngrams_report)
		self.corpus.results_cache[cache_id] = (ngrams_report, total_unique, total_count)
	
	resultset_start = page_size*page_current
	resultset_end = page_size*(page_current+1)

	# get specific chunk of report into pandas based on resultset_start:
	ngrams_report_page = ngrams_report.slice(resultset_start, page_size).unnest('ngram_token_ids')
	token_strs = []
	for i in range(ngram_length):
		token_strs.append(self.corpus.token_ids_to_tokens(ngrams_report_page[f'token_{i+1}'].to_numpy()))
	token_strs = np.array(token_strs)
	ngram_text = [' '.join(column) for column in token_strs.T]
	ngrams_report_page = ngrams_report_page.with_columns(pl.Series(name="ngram", values=ngram_text))
	#ngrams_report_page = ngrams_report_page.to_pandas().set_index('rank')
	total_pages = math.ceil(total_unique/page_size)
	summary_data = {'ngram_length': ngram_length, 'ngram_word_position': ngram_word_position, 'total_unique': total_unique, 'total_count': total_count, 'page_current': page_current, 'total_pages': total_pages}
	formatted_data = [f'Total unique ngrams: {total_unique:,}', f'Total ngrams: {total_count:,}', f'Showing {min(page_size, total_count)} rows', f'Page {page_current+1} of {total_pages}'] 

	if show_all_columns == False:
		ngrams_report_page = ngrams_report_page[['ngram', 'frequency']]
	
	logger.info(f'Ngrams report time: {(time.time() - start_time):.5f} seconds')

	return Result(type = 'ngrams', df=ngrams_report_page, title=f'Ngrams for "{token_str}"', description=f'Ngram length: {ngram_length}, Token position: {ngram_word_position.lower()}', summary_data=summary_data, formatted_data=formatted_data)


In [ ]:
# run the ngrams method and display the results
report_brown.ngrams('i', ngram_length = 2, ngram_word_position = 'LEFT').display()


Ngrams for "i" 
 
 
 Ngram length: 2, Token position: right 
 
 
 Ngram 
 Frequency 
 
 
 
 
 , i 
 820 
 
 
 . i 
 783 
 
 
 and i 
 303 
 
 
 ` i 
 220 
 
 
 that i 
 164 
 
 
 but i 
 126 
 
 
 when i 
 111 
 
 
 as i 
 108 
 
 
 if i 
 96 
 
 
 ? i 
 85 
 
 
 what i 
 78 
 
 
 which i 
 68 
 
 
 -- i 
 49 
 
 
 because i 
 33 
 
 
 think i 
 32 
 
 
 time i 
 30 
 
 
 before i 
 27 
 
 
 where i 
 25 
 
 
 while i 
 22 
 
 
 all i 
 21 
 
 
 
 
 
 Total unique ngrams: 484 
 


 
 Total ngrams: 4,366 
 


 
 Showing 20 rows 
 


 
 Page 1 of 25

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()